In [1]:
import os
os.chdir(r'c:\Users\Ken\Documents\GitHub\coursera_rag')


In [2]:
import pandas as pd
df = pd.read_csv('requirements_from_acts.csv')
df.fillna('blank', inplace=True)  # remove any NaN values as it blows up serialization
data = df.to_dict('records')
len(data)

C:\Users\Ken\AppData\Local\Temp\ipykernel_12996\2978276498.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'blank' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('blank', inplace=True)  # remove any NaN values as it blows up serialization


227

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ken\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [9]:
# Create collection to store records
qdrant.recreate_collection(
    collection_name="legal_reqts",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\Ken\AppData\Local\Temp\ipykernel_12996\2027838554.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [13]:
# vectorize
qdrant.upload_points(
    collection_name="legal_reqts",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Essential Requirement"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the records
    ]
)

In [14]:
user_prompt = "Identify all requirements for risk analysis"

In [15]:
# Search source

hits = qdrant.search(
    collection_name="legal_reqts",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'ID': 69, 'Document ID': '2006/42/EC', 'Document Title': 'Machinery Directive', 'Annex': 'I', 'Section L1': 1, 'Section L2': 1.0, 'Section L3': 2.0, 'Section L4': '(b)', 'Essential Requirement': '1.1.2.(b) In selecting the most appropriate methods, the manufacturer or his authorised representative must apply the following principles, in the order given: — eliminate or reduce risks as far as possible (inherently safe machinery design and construction), — take the necessary protective measures in relation to risks that cannot be eliminated, — inform users of the residual risks due to any shortcomings of the protective measures adopted, indi\x02cate whether any particular training is required and specify any need to provide personal protective equipment.'} score: 0.5584695383993481
{'ID': 136, 'Document ID': '2006/42/EC', 'Document Title': 'Machinery Directive', 'Annex': 'I', 'Section L1': 1, 'Section L2': 7.0, 'Section L3': 2.0, 'Section L4': 'blank', 'Essential Requirement': '1.7.2. Wa

In [16]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [17]:
# Connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ModuleNotFoundError: No module named 'openai'